In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/comp9517/Aerial_Landscapes /content/
img_dir = "/content/Aerial_Landscapes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
res_dir = '/content/drive/MyDrive/comp9517/res'
os.makedirs(res_dir, exist_ok=True)

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# sampling

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def generate_train_test_powerlaw_indices(
    img_dir,
    alpha=1.0,
    train_ratio=0.8,
    seed=42,
    plot=True
):
    np.random.seed(seed)
    class_names = sorted([
        d for d in os.listdir(img_dir)
        if os.path.isdir(os.path.join(img_dir, d))
    ])

    train_indices_dict = {}
    test_indices_dict = {}
    original_train_dict = {}

    global_idx = 0  # Track dataset-wide index

    for cls_idx, cls in enumerate(class_names):
        class_dir = os.path.join(img_dir, cls)
        image_files = sorted([
            f for f in os.listdir(class_dir)
            if f.lower().endswith(('.jpg', '.png'))
        ])
        class_len = len(image_files)
        class_indices = list(range(global_idx, global_idx + class_len))

        train_idx, test_idx = train_test_split(
            class_indices, train_size=train_ratio, random_state=seed, shuffle=True
        )

        # Save unmodified train/test
        original_train_dict[cls_idx] = train_idx
        test_indices_dict[cls_idx] = test_idx

        # Power-law on train only
        rank = cls_idx + 1
        weight = (1 / (rank ** alpha))
        weight /= (1 / (1 ** alpha))

        n_sample = int(len(train_idx) * weight)
        sampled = np.random.choice(train_idx, size=n_sample, replace=False)
        train_indices_dict[cls_idx] = sorted(sampled.tolist())

        global_idx += class_len

    if plot:
        _plot_sample_distribution(train_indices_dict, title=f"Train Distribution After Power-Law (α = {alpha})")
        _plot_sample_distribution(test_indices_dict, title="Test Distribution (Unmodified)")

    return train_indices_dict, test_indices_dict, original_train_dict

def _plot_sample_distribution(index_dict, title="Sample Distribution"):
    class_names = list(index_dict.keys())
    counts = [len(index_dict[cls]) for cls in class_names]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_names, counts, color='slateblue')
    plt.xticks(rotation=45, ha='right')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Number of Samples")
    plt.tight_layout()
    plt.grid(axis='y')

    for bar, count in zip(bars, counts):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 5,
            str(count),
            ha='center',
            va='bottom',
            fontsize=9
        )
    plt.savefig(os.path.join(res_dir, f"{title}.png"))
    plt.close()

In [5]:
# moderate a = 1
moderate_train_dict, moderate_test_dict, _ = generate_train_test_powerlaw_indices(
    img_dir, alpha=1, plot=True
)

# severe a = 1.5
severe_train_dict, severe_test_dict, _ = generate_train_test_powerlaw_indices(
    img_dir, alpha=1.5, plot=True
)

In [6]:
import os
from torch.utils.data import Subset

# Converts a train/test index dictionary into a pytorch subset
def get_subset_from_index_dict(dataset, index_dict, root_dir):
    path_to_index = {
        os.path.relpath(path, root_dir): i
        for i, (path, _) in enumerate(dataset.samples)
    }

    selected_indices = []
    for class_name, indices in index_dict.items():
        for idx in indices:
            rel_path = os.path.join(class_name, f"{idx:03d}.jpg")
            dataset_idx = path_to_index.get(rel_path)
            if dataset_idx is not None:
                selected_indices.append(dataset_idx)

    return Subset(dataset, selected_indices)

# resnet

In [7]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

def resnet18_train_baseline(train_indices_dict, test_indices_dict,
                            num_classes=15, epoch=5, batch_size=32,
                            device='cuda', res_dir='results'):

    weights = ResNet18_Weights.DEFAULT
    preprocess = weights.transforms()

    # dataset and subset construction
    full_dataset = datasets.ImageFolder(root=img_dir, transform=preprocess)
    imbal_train_indices = [i for indices in train_indices_dict.values() for i in indices]
    test_indices = [i for indices in test_indices_dict.values() for i in indices]

    imbal_train_dataset = Subset(full_dataset, imbal_train_indices)
    test_dataset = Subset(full_dataset, test_indices)

    train_loader = DataLoader(imbal_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # model setup
    model = resnet18(weights=weights)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # training
    for ep in range(epoch):
        model.train()
        total_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"[Epoch {ep+1}] Training Loss: {total_loss:.4f}")

    # evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    class_names = full_dataset.classes
    print("Classification Report (ResNet18 baseline):")
    print(classification_report(
        y_true, y_pred,
        labels=list(range(len(class_names))),
        target_names=class_names,
        digits=4,
        zero_division=0
    ))

    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

    title = "ResNet18 Baseline"
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix: {title}")
    plt.tight_layout()
    os.makedirs(res_dir, exist_ok=True)
    plt.savefig(f'{res_dir}/{title.replace(" ", "_")}.png')
    plt.close()

In [8]:
print("resnet18 on moderate imbalanced data ...")
resnet18_train_baseline(moderate_train_dict, moderate_test_dict)

print("resnet18 on severe imbalanced data ...")
resnet18_train_baseline(severe_train_dict, severe_test_dict)

resnet18 on moderate imbalanced data ...
[Epoch 1] Training Loss: 50.9024
[Epoch 2] Training Loss: 10.1875
[Epoch 3] Training Loss: 4.0619
[Epoch 4] Training Loss: 2.6256
[Epoch 5] Training Loss: 3.4289
Classification Report (ResNet18 baseline):
              precision    recall  f1-score   support

 Agriculture     0.7900    0.9875    0.8778       160
     Airport     0.7321    0.9563    0.8293       160
       Beach     0.9034    0.9938    0.9464       160
        City     0.8579    0.9812    0.9155       160
      Desert     0.9444    0.9563    0.9503       160
      Forest     0.8939    1.0000    0.9440       160
   Grassland     0.9441    0.9500    0.9470       160
     Highway     0.9667    0.9062    0.9355       160
        Lake     0.9510    0.8500    0.8977       160
    Mountain     0.9463    0.8812    0.9126       160
     Parking     0.9684    0.9563    0.9623       160
        Port     1.0000    0.8625    0.9262       160
     Railway     0.9741    0.7063    0.8188       1

# efficientnet_b0

In [9]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

def efficientnet_b0_train_baseline(train_indices_dict, test_indices_dict,
                                   num_classes=15, epoch=5, batch_size=32,
                                   device='cuda', res_dir='results'):

    weights = EfficientNet_B0_Weights.DEFAULT
    preprocess = weights.transforms()

    # Dataset and subset construction
    full_dataset = datasets.ImageFolder(root=img_dir, transform=preprocess)
    imbal_train_indices = [i for indices in train_indices_dict.values() for i in indices]
    test_indices = [i for indices in test_indices_dict.values() for i in indices]

    imbal_train_dataset = Subset(full_dataset, imbal_train_indices)
    test_dataset = Subset(full_dataset, test_indices)

    train_loader = DataLoader(imbal_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # Model setup
    model = efficientnet_b0(weights=weights)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Training loop
    for ep in range(epoch):
        model.train()
        total_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"[Epoch {ep+1}] Training Loss: {total_loss:.4f}")

    # Evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    class_names = full_dataset.classes
    print("Classification Report (EfficientNet_B0 baseline):")
    print(classification_report(
        y_true, y_pred,
        labels=list(range(len(class_names))),
        target_names=class_names,
        digits=4,
        zero_division=0
    ))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

    title = "EfficientNet_B0 Baseline"
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix: {title}")
    plt.tight_layout()
    os.makedirs(res_dir, exist_ok=True)
    plt.savefig(f'{res_dir}/{title.replace(" ", "_")}.png')
    plt.close()

In [10]:
print("efficientnet_b0 on moderate imbalanced data ...")
efficientnet_b0_train_baseline(moderate_train_dict, moderate_test_dict)

print("efficientnet_b0 on severe imbalanced data ...")
efficientnet_b0_train_baseline(severe_train_dict, severe_test_dict)

efficientnet_b0 on moderate imbalanced data ...
[Epoch 1] Training Loss: 125.2052
[Epoch 2] Training Loss: 53.8497
[Epoch 3] Training Loss: 29.5112
[Epoch 4] Training Loss: 15.1639
[Epoch 5] Training Loss: 10.0199
Classification Report (EfficientNet_B0 baseline):
              precision    recall  f1-score   support

 Agriculture     0.8674    0.9812    0.9208       160
     Airport     0.8486    0.9812    0.9101       160
       Beach     0.9080    0.9875    0.9461       160
        City     0.8977    0.9875    0.9405       160
      Desert     0.9333    0.9625    0.9477       160
      Forest     0.8290    1.0000    0.9065       160
   Grassland     0.9787    0.8625    0.9169       160
     Highway     0.9481    0.9125    0.9299       160
        Lake     0.9267    0.8688    0.8968       160
    Mountain     0.9396    0.8750    0.9061       160
     Parking     0.9810    0.9688    0.9748       160
        Port     0.9677    0.9375    0.9524       160
     Railway     0.9209    0.8000

# Performance Boosting

In [11]:
import torch.nn.functional as F

def focal_loss(outputs, targets, alpha=None, gamma=2.0):
    ce_loss = F.cross_entropy(outputs, targets, reduction='none')
    pt = torch.exp(-ce_loss)
    alpha_t = alpha[targets] if alpha is not None else 1.0
    return (alpha_t * (1 - pt) ** gamma * ce_loss).mean()

def dice_loss(outputs, targets, smooth=1e-5):
    num_classes = outputs.size(1)
    outputs = F.softmax(outputs, dim=1)

    targets_onehot = F.one_hot(targets, num_classes=num_classes).float()

    intersection = (outputs * targets_onehot).sum(dim=0)
    union = outputs.sum(dim=0) + targets_onehot.sum(dim=0)

    dice = (2 * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

In [12]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn.functional as F

def remix_batch(x, y, class_counts, tau=0.5, kappa=0.8, num_classes=15):
    batch_size = x.size(0)
    idx_perm = torch.randperm(batch_size)
    x2, y2 = x[idx_perm], y[idx_perm]

    probs = torch.tensor([class_counts.get(int(label.item()), 1) for label in y], dtype=torch.float32)
    probs_perm = torch.tensor([class_counts.get(int(label.item()), 1) for label in y2], dtype=torch.float32)

    probs = probs / probs.sum()
    probs_perm = probs_perm / probs_perm.sum()

    lam = np.random.beta(1.0, 1.0)
    remix_mask = (probs_perm < kappa * probs).float()
    lam_adj = remix_mask * tau + (1 - remix_mask) * lam

    lam_adj = lam_adj.view(-1, 1, 1, 1).to(x.device)
    x_mix = lam_adj * x + (1 - lam_adj) * x2

    lam_adj_flat = remix_mask * tau + (1 - remix_mask) * lam
    y_onehot = F.one_hot(y, num_classes=num_classes).float()
    y2_onehot = F.one_hot(y2, num_classes=num_classes).float()
    y_mix = lam_adj_flat.view(-1, 1).to(x.device) * y_onehot + (1 - lam_adj_flat.view(-1, 1)).to(x.device) * y2_onehot

    return x_mix, y_mix

def resnet18_train(train_indices_dict, test_indices_dict, loss_type='cross_entropy',
                   alpha=None, gamma=2.0, epoch=5,
                   lambda_focal=1.0, lambda_dice=0.5,
                   augmentation=False, useSampler=False, remix=False,
                   rotation_deg=15, crop_scale=(0.8, 1.0),
                   tau=0.5, kappa=0.8,
                   num_classes=15, batch_size=32, device='cuda', res_dir=res_dir):

    weights = ResNet18_Weights.DEFAULT
    preprocess = weights.transforms()
    image_size = weights.transforms().crop_size[0]

    data_augmentation = transforms.Compose([
        transforms.RandomResizedCrop(image_size, scale=crop_scale),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=weights.transforms().mean, std=weights.transforms().std)
    ])

    raw_dataset = datasets.ImageFolder(root=img_dir)
    transform_train_dataset = datasets.ImageFolder(root=img_dir, transform=data_augmentation if augmentation else preprocess)
    imbal_train_indices = [i for indices in train_indices_dict.values() for i in indices]
    imbal_train_dataset = Subset(transform_train_dataset, imbal_train_indices)

    train_targets = [raw_dataset.targets[i] for i in imbal_train_indices]
    class_counts = Counter(train_targets)

    if alpha is None and 'focal' in loss_type:
        beta = 0.999
        alpha_vals = []
        for i in range(num_classes):
            count = class_counts.get(i, 1)
            eff_num = (1 - beta ** count) / (1 - beta)
            alpha_vals.append(1 / eff_num)
        alpha_tensor = torch.tensor(alpha_vals, dtype=torch.float32)
        alpha_tensor = alpha_tensor / alpha_tensor.sum()
        alpha = alpha_tensor.to(device)

    if useSampler:
        class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
        sample_weights = [class_weights[label] for label in train_targets]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        train_loader = DataLoader(imbal_train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4)
    else:
        train_loader = DataLoader(imbal_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    transform_test_dataset = datasets.ImageFolder(root=img_dir, transform=preprocess)
    test_indices = [i for indices in test_indices_dict.values() for i in indices]
    test_dataset = Subset(transform_test_dataset, test_indices)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    model = resnet18(weights=weights)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for ep in range(epoch):
        model.train()
        total_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()

            if remix:
                imgs, labels_soft = remix_batch(imgs, labels, class_counts, tau=tau, kappa=kappa, num_classes=num_classes)
                outputs = model(imgs)
                log_probs = F.log_softmax(outputs, dim=1)
                remix_loss = -(labels_soft * log_probs).sum(dim=1).mean()
                hard_labels = labels_soft.argmax(dim=1)

                if loss_type == 'focal':
                    loss = remix_loss + lambda_focal * focal_loss(outputs, hard_labels, alpha=alpha, gamma=gamma)
                elif loss_type == 'dice':
                    loss = remix_loss + lambda_dice * dice_loss(outputs, hard_labels)
                elif loss_type == 'focal+dice':
                    loss = remix_loss + lambda_focal * focal_loss(outputs, hard_labels, alpha=alpha, gamma=gamma) + lambda_dice * dice_loss(outputs, hard_labels)
                else:
                    loss = remix_loss
            else:
                outputs = model(imgs)
                if loss_type == 'focal':
                    loss = focal_loss(outputs, labels, alpha=alpha, gamma=gamma)
                elif loss_type == 'dice':
                    loss = dice_loss(outputs, labels)
                elif loss_type == 'focal+dice':
                    loss = lambda_focal * focal_loss(outputs, labels, alpha=alpha, gamma=gamma) + lambda_dice * dice_loss(outputs, labels)
                else:
                    loss = nn.CrossEntropyLoss()(outputs, labels)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"[Epoch {ep+1}] Training Loss: {total_loss:.4f}")

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    class_names = transform_test_dataset.classes
    title = f"ResNet18 - {loss_type}" + (" + aug" if augmentation else "") + (" + sampler" if useSampler else "") + (" + remix" if remix else "")
    print(f"Classification Report: {title}")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix: {title}")
    plt.tight_layout()
    os.makedirs(res_dir, exist_ok=True)
    plt.savefig(f'{res_dir}/{title.replace(" ", "_").replace("+", "plus")}.png')
    plt.close()

In [13]:
print("resnet18 moderate imbalanced data")
resnet18_train(moderate_train_dict, moderate_test_dict)

print("resnet18 moderate imbalanced data - focal ...")
resnet18_train(moderate_train_dict, moderate_test_dict, loss_type='focal')

print("resnet18 moderate imbalanced data - focal + dice ...")
resnet18_train(moderate_train_dict, moderate_test_dict, loss_type='focal+dice')

print("resnet18 moderate imbalanced data - focal + dice + sampler ...")
resnet18_train(moderate_train_dict, moderate_test_dict, loss_type='focal+dice', useSampler=True)

print("resnet18 moderate imbalanced data - focal + dice + augmentation ...")
resnet18_train(moderate_train_dict, moderate_test_dict, loss_type='focal+dice', augmentation=True)

print("resnet18 moderate imbalanced data - focal + dice + augmentation + sampler ...")
resnet18_train(moderate_train_dict, moderate_test_dict, loss_type='focal+dice', augmentation=True, useSampler=True)

resnet18 moderate imbalanced data
[Epoch 1] Training Loss: 56.3022
[Epoch 2] Training Loss: 9.1129
[Epoch 3] Training Loss: 3.7320
[Epoch 4] Training Loss: 2.8800
[Epoch 5] Training Loss: 2.6117
Classification Report: ResNet18 - cross_entropy
              precision    recall  f1-score   support

 Agriculture     0.8290    1.0000    0.9065       160
     Airport     0.6515    0.9812    0.7830       160
       Beach     0.9034    0.9938    0.9464       160
        City     0.8814    0.9750    0.9258       160
      Desert     0.9387    0.9563    0.9474       160
      Forest     0.8814    0.9750    0.9258       160
   Grassland     0.9503    0.9563    0.9533       160
     Highway     0.9542    0.9125    0.9329       160
        Lake     0.9718    0.8625    0.9139       160
    Mountain     0.9783    0.8438    0.9060       160
     Parking     1.0000    0.9313    0.9644       160
        Port     0.9795    0.8938    0.9346       160
     Railway     0.9490    0.5813    0.7209       160


In [14]:
print("resnet18 severe imbalanced data")
resnet18_train(severe_train_dict, severe_test_dict)

print("resnet18 severe imbalanced data - focal ...")
resnet18_train(severe_train_dict, severe_test_dict, loss_type='focal', gamma=1.5)

print("resnet18 severe imbalanced data - focal + dice ...")
resnet18_train(severe_train_dict, severe_test_dict, loss_type='focal+dice', gamma=1.5)

print("resnet18 severe imbalanced data - focal + dice + sampler ...")
resnet18_train(severe_train_dict, severe_test_dict, loss_type='focal+dice', useSampler=True, gamma=1.5)

print("resnet18 severe imbalanced data - focal + dice + augmentation ...")
resnet18_train(severe_train_dict, severe_test_dict, loss_type='focal+dice', augmentation=True, gamma=1.5)

print("resnet18 severe imbalanced data - focal + dice + augmentation + sampler ...")
resnet18_train(severe_train_dict, severe_test_dict, loss_type='focal+dice', augmentation=True, useSampler=True, gamma=1.5)

resnet18 severe imbalanced data
[Epoch 1] Training Loss: 38.8698
[Epoch 2] Training Loss: 7.4976
[Epoch 3] Training Loss: 3.0806
[Epoch 4] Training Loss: 1.5237
[Epoch 5] Training Loss: 0.8865
Classification Report: ResNet18 - cross_entropy
              precision    recall  f1-score   support

 Agriculture     0.6987    1.0000    0.8226       160
     Airport     0.5200    0.9750    0.6783       160
       Beach     0.7949    0.9688    0.8732       160
        City     0.7277    0.9688    0.8311       160
      Desert     0.9217    0.9563    0.9387       160
      Forest     0.8413    0.9938    0.9112       160
   Grassland     0.8944    0.9000    0.8972       160
     Highway     0.8940    0.8438    0.8682       160
        Lake     0.9286    0.7312    0.8182       160
    Mountain     0.8256    0.8875    0.8554       160
     Parking     0.9724    0.8812    0.9246       160
        Port     0.9318    0.7688    0.8425       160
     Railway     0.8393    0.2938    0.4352       160
 R